In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.metrics import mean_squared_error


In [12]:

thickness_lst = []
wtw_thickness = []
refractive_lst = []
wtw_refractive = []
f = pd.ExcelFile('Data/O3 development.xlsx')

for name in f.sheet_names:
    if name == 'DOE':
        doe = pd.read_excel('Data/O3 development.xlsx', engine='openpyxl',sheet_name = 'DOE')
        doe = doe[doe.columns[-6::]]
        doe.columns = [['Ar (sccm)', 'O3 (sccm)', 'Turn on (s) TMA', 'Turn off (s) TMA','Turn off (s) Ozone','Turn off (s) Ozone']]
        doe = doe[1:]
        doe.index = ['Run{}'.format(x) for x in range(15,23)]
    else:
        cur_sheet = pd.read_excel('Data/O3 development.xlsx', engine='openpyxl',sheet_name = name)
        needed_cols = ['Unnamed: {}'.format(x) for x in range(2,13)]
        thickness = cur_sheet[needed_cols][2:26].reset_index(drop = True)
        thickness.columns = ['SE testing point:{}'.format(x) for x in range(1, 10)]+['Avg', 'WIW']
        thickness_lst.append(thickness)
        
        refractive = cur_sheet[needed_cols][30::].reset_index(drop = True)
        refractive.columns = ['SE testing point:{}'.format(x) for x in range(1, 10)] + ['Avg','WIW']
        refractive_lst.append(refractive)
        
        wtw_thickness.append(cur_sheet['Unnamed: 13'].iloc[2])
        wtw_refractive.append(cur_sheet['Unnamed: 13'].iloc[30])

In [13]:
para = doe.to_numpy()
# rf = np.array(refractive_lst)
rf_wtw = np.array(wtw_refractive)
# tk = np.array(thickness_lst)
tk_wtw = np.array(wtw_thickness)

In [15]:
### Data setup 
NUM_OF_DATA_POINTS = 20
NUM_OF_PLOT_POINTS = 100
# gp settings 
NOISE_LEVEL = 1e-5

kernel = 1.0 * RBF(length_scale=5) + WhiteKernel(noise_level=NOISE_LEVEL)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-8)

In [26]:
X = para 
Y = np.array([tk_wtw,rf_wtw]).reshape(8,2)

In [27]:
#### Kernel setting and prediction
gp.fit(X, Y)
print("Learned kernel", gp.kernel_)

Learned kernel 0.0537**2 * RBF(length_scale=783) + WhiteKernel(noise_level=1.96e-05)


In [57]:
para_ar = 2500
para_o3 = 3300
para_TMA_on = 5
para_TMA_off = 7
para_Ozone_on = 7
para_Ozone_off = 8
X_predict = np.array([para_ar,para_o3,para_TMA_on,para_TMA_off,
                        para_Ozone_on, para_Ozone_off]).reshape(1,6)

Y_predict = gp.predict(X_predict)
print('WIW:thickness %s, n %s'%(Y_predict[:,0],Y_predict[:,1]))

WIW:thickness [0.00257006], n [0.00069536]
